<a href="https://colab.research.google.com/github/NdumbiData/Fraudulent-Transaction-Detection-for-Digital-Money-Transfer/blob/main/Cleaning_The_Data_set_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fraudulent Transaction Detection for Digital Money Transfer

#Cleaning the Dataset

Import libriaries

In [ ]:
# Data analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Import the dataset

df = pd.read_csv("/content/drive/MyDrive/Data Science Internship/nova_pay_combined.csv")

In [ ]:
df.head(2)


,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,...,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
0,fee8542d-8ee6-4b0d-9671-c294dd08ed26,402cccc9-28de-45b3-9af7-cc5302aa1f93,2022-10-03 18:40:59.468549+00:00,US,USD,CAD,ATM,278.19,278.19,4.25,...,0.123,standard,263,0.522,0,0.223,0,0,0.0,0
1,bfdb9fc1-27fe-4a85-b043-4d813d679259,67c2c6b3-ef0a-4777-a3f1-c84a851bb6ad,2022-10-03 20:39:38.468549+00:00,CA,CAD,MXN,web,208.51,154.29,4.24,...,0.569,standard,947,0.475,0,0.268,0,1,0.0,0


## Date Cleaning

*   Ensures the data is clean and features have the correct data

In [ ]:
##Change data types to reflect the features correctly

df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

df["amount_src"] = pd.to_numeric(df["amount_src"],  errors='coerce')

# Fill missing values in 'amount_src' with the median
df["amount_src"] = df["amount_src"].fillna(df["amount_src"].median())

In [ ]:

df["timestamp"].isnull().sum()

np.int64(0)

In [ ]:
df.dropna(subset=["timestamp"], inplace=True)

In [ ]:
df["timestamp"].isnull().sum()

np.int64(0)

 Calculation of exchange rates per currency



* : Select rows where Amount USD is present<br>

* : Group by source currency <br>

* : Computes the mean of Amount usd /amount_src for each currency . <br>

* : Converts the result to a dictionary for easy lookup <br>

In [ ]:
exchange_rates = df[df["amount_usd"].notna()].groupby("source_currency").apply(
    lambda x: (x["amount_usd"] / x["amount_src"]).mean()
).to_dict()


print(exchange_rates)

{'CAD': 0.7215175313268939, 'GBP': 1.2608696002479334, 'USD': 1.0008316967687891}


/tmp/ipython-input-3989434759.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exchange_rates = df[df["amount_usd"].notna()].groupby("source_currency").apply(


In [ ]:
df["source_currency"].value_counts()

,count
source_currency,
USD,7988
GBP,2127
CAD,1224


This Cells fills missing **amount_USD** Values:

* If **amount_USD** is already present , Keep it <br>

* Otherwise ,calculate it using the **amount_src** multiplied by the echange rate for that **source_currency**.<br>

* Defaults to 1 if the currency is not in exchange_rates.

In [ ]:
df["amount_usd"] = df.apply(
    lambda row: row["amount_src"] * exchange_rates.get(row["source_currency"], 1) if pd.isna(row["amount_usd"]) else row["amount_usd"],
    axis=1
)

In [ ]:
df['fee'].head()

,fee
0,4.25
1,4.24
2,2.70
3,2.22
4,3.61


This cell fills missing fee values

* If channel exists , fill missing values per channels median
* Then fill the rest using the overal median

In [ ]:
#fee: The median or by channel present

if "fee" in df.columns:
  if 'channel' in df.columns:
    df['fee'] = df.groupby('channel')['fee'].transform(lambda s: s.fillna(s.median()))
  df['fee']=df['fee'].fillna(df['fee'].median())

  print(df['fee'])

0         4.25
1         4.24
2         2.70
3         2.22
4         3.61
         ...  
11395     5.24
11396     8.86
11397     4.00
11398     1.96
11399    19.34
Name: fee, Length: 11339, dtype: float64


In [ ]:
print(df.isnull().sum())

transaction_id                 0
customer_id                    0
timestamp                      0
home_country                   0
source_currency                0
dest_currency                  0
channel                        0
amount_src                     0
amount_usd                     0
fee                            0
exchange_rate_src_to_dest      0
device_id                      0
new_device                     0
ip_address                   305
ip_country                   301
location_mismatch              0
ip_risk_score                  0
kyc_tier                     300
account_age_days               0
device_trust_score           295
chargeback_history_count       0
risk_score_internal            0
txn_velocity_1h                0
txn_velocity_24h               0
corridor_risk                  0
is_fraud                       0
dtype: int64


In [ ]:
print(df['amount_usd'].isnull().sum())

0


In [ ]:
# ip_country: fallback to home country

if {'ip_country', 'home_country'}.issubset(df.columns):
  df['ip_country'] = df['ip_country'].fillna(df['home_country'])

  print(df['ip_country'])

0        US
1        CA
2        US
3        US
4        US
         ..
11395    US
11396    UK
11397    US
11398    US
11399    US
Name: ip_country, Length: 11339, dtype: object


This cell fills missing kyc_tier values:

* Find the most frequent kyc_tier values
* If the mode is unavailable defaults to "standard"
* Fills missing values with this model/default.

In [ ]:
# Kyc Tier: Fills with mode
if "kyc_tier" in df.columns:
  mode_kyc = df["kyc_tier"].mode()[0] if not df["kyc_tier"].mode().empty else "standard"
  df["kyc_tier"] = df["kyc_tier"].fillna(mode_kyc)
  print(df["kyc_tier"])

0        standard
1        standard
2        enhanced
3        standard
4        enhanced
           ...   
11395    standard
11396         low
11397         low
11398    enhanced
11399         low
Name: kyc_tier, Length: 11339, dtype: object


This cell fills missing device_trust_score values:

* If new device and kyc tier exists, fill missing score per group using the group's median.

* Then fill any remaining missing scores with the overal median.

In [ ]:
if "device_trust_score" in df.columns:
  if {'new_device','kyc_tier'}.issubset(df.columns):
    df['device_trust_score'] = df.groupby(['new_device', 'kyc_tier'])['device_trust_score'].transform(lambda s: s.fillna(s.median()))
  df['device_trust_score'] = df['device_trust_score'].fillna(df['device_trust_score'].median())

  print(df['device_trust_score'])

0        0.522
1        0.475
2        0.939
3        0.551
4        0.894
         ...  
11395    0.653
11396    0.173
11397    0.269
11398    0.773
11399    0.187
Name: device_trust_score, Length: 11339, dtype: float64


In [ ]:
print(df.isnull().sum())

transaction_id                 0
customer_id                    0
timestamp                      0
home_country                   0
source_currency                0
dest_currency                  0
channel                        0
amount_src                     0
amount_usd                     0
fee                            0
exchange_rate_src_to_dest      0
device_id                      0
new_device                     0
ip_address                   305
ip_country                     0
location_mismatch              0
ip_risk_score                  0
kyc_tier                       0
account_age_days               0
device_trust_score             0
chargeback_history_count       0
risk_score_internal            0
txn_velocity_1h                0
txn_velocity_24h               0
corridor_risk                  0
is_fraud                       0
dtype: int64


In [ ]:
output_path = '/content/drive/MyDrive/Data Science Internship/cleaned_data_nova_pay_Combined.csv'
df.to_csv(output_path, index=False)